In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os

In [43]:
file_path = os.path.join('data', 'transfer_death.csv')

patients_transfer = pd.read_csv(file_path)
patients_transfer.careunit = patients_transfer.careunit.ffill()
patients_transfer.head()

,subject_id,gender,anchor_age,careunit,dod
0,10030412,F,38,Emergency Department,NaN
1,10030412,F,38,Emergency Department,NaN
2,10030412,F,38,Medicine,NaN
3,10030412,F,38,Emergency Department,NaN
4,10030412,F,38,Emergency Department,NaN


In [44]:
# Totol transfer for each patient
transfer_times = patients_transfer.loc[:, ['subject_id', 'careunit']].groupby('subject_id').count()
transfer_times.head()

,careunit
subject_id,
10018852,9
10024331,126
10025647,68
10029206,3
10030412,6


In [45]:
# Each type of transfer for each patient
each_transfer_times = patients_transfer.loc[:, ['subject_id', 'careunit', 'gender']].groupby(by=['subject_id', 'careunit']).count()
each_transfer_times.head()

gender
subject_id careunit                        
10018852   Discharge Lounge               3
           Emergency Department           2
           Medicine                       4
10024331   Coronary Care Unit (CCU)       2
           Emergency Department          40

In [46]:
careunit_dict = patients_transfer.careunit.value_counts()
careunit_dict.index

Index(['Emergency Department', 'Medicine', 'Transplant', 'Med/Surg',
       'Hematology/Oncology', 'Discharge Lounge',
       'Emergency Department Observation', 'Medical/Surgical (Gynecology)',
       'Medicine/Cardiology', 'Med/Surg/GYN', 'Vascular', 'Neurology',
       'Medical Intensive Care Unit (MICU)', 'PACU', 'Med/Surg/Trauma',
       'Surgery/Pancreatic/Biliary/Bariatric',
       'Surgical Intensive Care Unit (SICU)',
       'Medical/Surgical Intensive Care Unit (MICU/SICU)', 'Surgery/Trauma',
       'Hematology/Oncology Intermediate', 'Cardiac Surgery', 'Surgery',
       'Obstetrics (Postpartum & Antepartum)', 'Coronary Care Unit (CCU)',
       'Psychiatry', 'Labor & Delivery',
       'Cardiac Vascular Intensive Care Unit (CVICU)', 'Trauma SICU (TSICU)',
       'Observation', 'Thoracic Surgery', 'Medicine/Cardiology Intermediate',
       'Obstetrics Postpartum',
       'Neuro Surgical Intensive Care Unit (Neuro SICU)',
       'Obstetrics Antepartum', 'Cardiology',
       'Car

#### Categories made by ChatGPT

In [50]:
# Category by ChatGPT
categories = {
    'Emergency Department': ['Emergency Department', 'Emergency Department Observation'],
    'Medicine': ['Medicine', 'Medicine/Cardiology', 'Hematology/Oncology', 'Medicine/Cardiology Intermediate', 'Hematology/Oncology Intermediate', 'Medical/Surgical (Gynecology)', 'Medical/Surgical Intensive Care Unit (MICU/SICU)', 'Medical Intensive Care Unit (MICU)', 'Medical/Surgical (GYN)', 'Medical/Surgical/Trauma', 'Medical/Surgical Stepdown'],
    'Transplant': ['Transplant'],
    'Surgery': ['Surgery', 'Surgery/Pancreatic/Biliary/Bariatric', 'Surgical Intensive Care Unit (SICU)', 'Surgery/Trauma', 'Thoracic Surgery'],
    'Obstetrics': ['Obstetrics (Postpartum & Antepartum)', 'Obstetrics Postpartum', 'Obstetrics Antepartum'],
    'Cardiology': ['Cardiology', 'Cardiac Surgery', 'Cardiology Surgery Intermediate', 'Coronary Care Unit (CCU)', 'Cardiac Vascular Intensive Care Unit (CVICU)'],
    'Neurology': ['Neurology', 'Neuro Surgical Intensive Care Unit (Neuro SICU)', 'Neuro Intermediate', 'Neuro Stepdown'],
    'Psychiatry': ['Psychiatry'],
    'Other': ['Discharge Lounge', 'Vascular', 'PACU', 'Observation', 'Unknown']
}

reverse_categories = {}
for category, items in categories.items():
    for item in items:
        reverse_categories[item] = category

In [52]:
patients_transfer_ = patients_transfer.copy()
patients_transfer_['careunit_category'] = patients_transfer_.careunit.map(reverse_categories)
patients_transfer_.head()

,subject_id,gender,anchor_age,careunit,dod,careunit_category
0,10030412,F,38,Emergency Department,NaN,Emergency Department
1,10030412,F,38,Emergency Department,NaN,Emergency Department
2,10030412,F,38,Medicine,NaN,Medicine
3,10030412,F,38,Emergency Department,NaN,Emergency Department
4,10030412,F,38,Emergency Department,NaN,Emergency Department


In [72]:
groupby_careunit_category = patients_transfer_.loc[:, ['subject_id', 'careunit_category', 'gender']].groupby(by=['subject_id', 'careunit_category']).count()
unique_subject_id_list = patients_transfer_.loc[:, ['subject_id', 'gender']].groupby('subject_id').count().index.to_list()
col_idx = list(categories.keys())

groupby_careunit_category

sim_orig = pd.DataFrame(np.zeros((len(unique_subject_id_list), len(col_idx))), index=unique_subject_id_list, columns=col_idx)

for i, subject_id in enumerate(unique_subject_id_list):
    for j, careunit_category in enumerate(col_idx):
        if (subject_id, careunit_category) in groupby_careunit_category.index:
            sim_orig.loc[subject_id, careunit_category] = groupby_careunit_category.loc[(subject_id, careunit_category), 'gender'].item()

sim_orig

,Emergency Department,Medicine,Transplant,Surgery,Obstetrics,Cardiology,Neurology,Psychiatry,Other
10018852,2.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
10024331,46.0,58.0,0.0,2.0,0.0,2.0,0.0,0.0,14.0
10025647,32.0,22.0,0.0,4.0,0.0,6.0,0.0,0.0,4.0
10029206,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
10030412,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
19970491,26.0,0.0,0.0,30.0,0.0,0.0,0.0,0.0,12.0
19971180,34.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19971435,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19980334,27.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [73]:
sim_orig.mean(axis=0)

Emergency Department    20.159191
Medicine                19.476693
Transplant               3.966579
Surgery                  2.035180
Obstetrics               0.343887
Cardiology               0.764292
Neurology                1.027265
Psychiatry               0.226033
Other                    3.994723
dtype: float64